In [20]:
import cv2  
import numpy as np  
#import pdb  
#pdb.set_trace()#turn on the pdb prompt  
  
#read image  
img = cv2.imread('7elevenLogo.jpg',cv2.IMREAD_COLOR)  
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
cv2.threshold(gray,127,255,cv2.THRESH_BINARY)
cv2.imshow('origin',img); 
  
#SIFT  
detector = cv2.SIFT()  
keypoints = detector.detect(gray,None)  
img = cv2.drawKeypoints(gray,keypoints)  
# img = cv2.drawKeypoints(gray,keypoints,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)  
# print type(keypoints[0])
cv2.imshow('test',img);  
cv2.waitKey(0)  
cv2.destroyAllWindows()  

In [9]:
#read image  
img = cv2.imread('simitsu2.jpg',cv2.IMREAD_COLOR)  
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
cv2.threshold(gray,127,255,cv2.THRESH_BINARY)
cv2.imshow('origin',img); 
  
#SIFT  
detector = cv2.SIFT()  
keypoints = detector.detect(gray,None)  
# img = cv2.drawKeypoints(gray,keypoints)  
img = cv2.drawKeypoints(gray,keypoints,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)  
cv2.imshow('test',img);  
cv2.waitKey(0)  
cv2.destroyAllWindows()  

# 畫出match的畫

In [21]:
import numpy as np
import cv2

def drawMatches(img1, kp1, img2, kp2, matches):
    """
    My own implementation of cv2.drawMatches as OpenCV 2.4.9
    does not have this function available but it's supported in
    OpenCV 3.0.0

    This function takes in two images with their associated 
    keypoints, as well as a list of DMatch data structure (matches) 
    that contains which keypoints matched in which images.

    An image will be produced where a montage is shown with
    the first image followed by the second image beside it.

    Keypoints are delineated with circles, while lines are connected
    between matching keypoints.

    img1,img2 - Grayscale images
    kp1,kp2 - Detected list of keypoints through any of the OpenCV keypoint 
              detection algorithms
    matches - A list of matches of corresponding keypoints through any
              OpenCV keypoint matching algorithm
    """

    # Create a new output image that concatenates the two images together
    # (a.k.a) a montage
    rows1 = img1.shape[0]
    cols1 = img1.shape[1]
    rows2 = img2.shape[0]
    cols2 = img2.shape[1]

    out = np.zeros((max([rows1,rows2]),cols1+cols2,3), dtype='uint8')

    # Place the first image to the left
    out[:rows1,:cols1] = np.dstack([img1, img1, img1])

    # Place the next image to the right of it
    out[:rows2,cols1:] = np.dstack([img2, img2, img2])

    # For each pair of points we have between both images
    # draw circles, then connect a line between them
    for mat in matches:

        # Get the matching keypoints for each of the images
        img1_idx = mat.queryIdx
        img2_idx = mat.trainIdx

        # x - columns
        # y - rows
        (x1,y1) = kp1[img1_idx].pt
        (x2,y2) = kp2[img2_idx].pt

        # Draw a small circle at both co-ordinates
        # radius 4
        # colour blue
        # thickness = 1
        cv2.circle(out, (int(x1),int(y1)), 4, (255, 0, 0), 1)   
        cv2.circle(out, (int(x2)+cols1,int(y2)), 4, (255, 0, 0), 1)

        # Draw a line in between the two points
        # thickness = 1
        # colour blue
        cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (255, 0, 0), 1)


    # Show the image
    cv2.imshow('Matched Features', out)
    cv2.waitKey(0)
    cv2.destroyWindow('Matched Features')

    # Also return the image if you'd like a copy
    return out

## 比對兩張圖像

In [26]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img1 = cv2.imread('simitsu1.jpg',0)          # queryImage
img2 = cv2.imread('simitsu2.jpg',0) # trainImage

# Initiate SIFT detector
orb = cv2.ORB()

# find the keypoints and descriptors with SIFT
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)

In [27]:
# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors.
matches = bf.match(des1,des2)

# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)

# Draw first 10 matches.
img3 = drawMatches(img1,kp1,img2,kp2,matches[:10])

plt.imshow(img3),plt.show()

AttributeError: 'NoneType' object has no attribute 'shape'

# 利用sift進行比對

In [29]:
import cv2  
import scipy as sp  
  
img1 = cv2.imread('7elevenLogo.jpg',0) # queryImage  
img2 = cv2.imread('7elevenTest.jpg',0) # trainImage  
  
# Initiate SIFT detector  
sift = cv2.SIFT()  
  
# find the keypoints and descriptors with SIFT  
kp1, des1 = sift.detectAndCompute(img1,None)  
kp2, des2 = sift.detectAndCompute(img2,None)  
  
# FLANN parameters  
FLANN_INDEX_KDTREE = 0  
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)  
search_params = dict(checks=50)   # or pass empty dictionary  
flann = cv2.FlannBasedMatcher(index_params,search_params)  
matches = flann.knnMatch(des1,des2,k=2)  
  
print 'matches...',len(matches)  
# Apply ratio test  
good = []  
for m,n in matches:  
    if m.distance < 0.75*n.distance:  
        good.append(m)  
print 'good',len(good)  
# #####################################  
# visualization  
h1, w1 = img1.shape[:2]  
h2, w2 = img2.shape[:2]  
view = sp.zeros((max(h1, h2), w1 + w2, 3), sp.uint8)  
view[:h1, :w1, 0] = img1  
view[:h2, w1:, 0] = img2  
view[:, :, 1] = view[:, :, 0]  
view[:, :, 2] = view[:, :, 0]  
  
for m in good:  
    # draw the keypoints  
    # print m.queryIdx, m.trainIdx, m.distance  
    color = tuple([sp.random.randint(0, 255) for _ in xrange(3)])  
    #print 'kp1,kp2',kp1,kp2  
    cv2.line(view, (int(kp1[m.queryIdx].pt[0]), int(kp1[m.queryIdx].pt[1])) , (int(kp2[m.trainIdx].pt[0] + w1), int(kp2[m.trainIdx].pt[1])), color)  
  
cv2.imshow("view", view)  
cv2.waitKey()  

matches... 191
good 24


-1

# 利用scipy比對

In [15]:
#coding=utf-8 
import cv2 
import scipy as sp 
  
img1 = cv2.imread('7elevenLogo.jpg',0) # queryImage 
img2 = cv2.imread('7elevenTest.jpg',0) # trainImage 
  
# Initiate SIFT detector 
sift = cv2.SIFT() 
  
# find the keypoints and descriptors with SIFT 
kp1, des1 = sift.detectAndCompute(img1,None) 
kp2, des2 = sift.detectAndCompute(img2,None) 
  
# FLANN parameters 
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5) 
search_params = dict(checks=50)  # or pass empty dictionary 
flann = cv2.FlannBasedMatcher(index_params,search_params) 
matches = flann.knnMatch(des1,des2,k=2) 
  
print 'matches...',len(matches) 
# Apply ratio test 
good = [] 
for m,n in matches: 
    if m.distance < 0.75*n.distance: 
        good.append(m) 
print 'good',len(good) 
# ##################################### 
# visualization 
h1, w1 = img1.shape[:2] 
h2, w2 = img2.shape[:2] 
view = sp.zeros((max(h1, h2), w1 + w2, 3), sp.uint8) 
view[:h1, :w1, 0] = img1 
view[:h2, w1:, 0] = img2 
view[:, :, 1] = view[:, :, 0] 
view[:, :, 2] = view[:, :, 0] 
  
for m in good: 
    #draw the keypoints 
    # print m.queryIdx, m.trainIdx, m.distance 
    color = tuple([sp.random.randint(0, 255) for _ in xrange(3)]) 
    ##print 'kp1,kp2',kp1,kp2 
    cv2.line(view, (int(kp1[m.queryIdx].pt[0]), int(kp1[m.queryIdx].pt[1])) , (int(kp2[m.trainIdx].pt[0] + w1), int(kp2[m.trainIdx].pt[1])), color) 
  
cv2.imshow("view", view) 
cv2.waitKey() 

matches... 191
good 24


-1

# 儲存點特徵準備訓練

In [61]:
import cv2
import numpy as np
import cPickle

img = cv2.imread('car.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT()
kp = sift.detect(gray, None)
img = cv2.drawKeypoints(img, keypoints = kp)

index = []

for point in kp:
    temp = (point.pt, point.size, point.angle, point.response, point.octave, 
            point.class_id) 
    index.append(temp)

## Put the keypoints into a File

f = open("keypoints.txt", "w")
f.write(cPickle.dumps(index))
f.close()
cv2.imwrite('sift_keypoints.jpg', img)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: ..\..\..\modules\imgproc\src\color.cpp:3739: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor


# SURF

In [50]:
img = cv2.imread('7elevenLogo.jpg',0)
# Create SURF object. You can specify params here or later.
# Here I set Hessian Threshold to 400
surf = cv2.SURF(400)
#Find keypoints and descriptors directly
kp, des = surf.detectAndCompute(img,None)
len(kp)

# Check present Hessian threshold
print surf.hessianThreshold

# We set it to some 50000. Remember, it is just for representing in picture.
# In actual cases, it is better to have a value 300-500
surf.hessianThreshold = 5000

# Again compute keypoints and check its number.
kp, des = surf.detectAndCompute(img,None)

print len(kp)

img2 = cv2.drawKeypoints(img,kp,None,(255,0,0),4)
plt.imshow(img2),plt.show()

400.0
125


(<matplotlib.image.AxesImage at 0xb843438>, None)

In [32]:
# Check upright flag, if it False, set it to True
print surf.upright

surf.upright = True

# Recompute the feature points and draw it
kp = surf.detect(img,None)
img2 = cv2.drawKeypoints(img,kp,None,(255,0,0),4)

plt.imshow(img2),plt.show()

True


(<matplotlib.image.AxesImage at 0xb37cbe0>, None)

In [33]:
# Find size of descriptor
print surf.descriptorSize()

# That means flag, "extended" is False.
print surf.extended

# So we make it to True to get 128-dim descriptors.
# surf.extended = True
kp, des = surf.detectAndCompute(img,None)
print surf.descriptorSize()

print des.shape

128
True
128
(254L, 128L)


In [52]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img1 = cv2.imread('7elevenLogo.jpg',0)
img2 = cv2.imread('7elevenTest.jpg',0)
# Create SURF object. You can specify params here or later.
# Here I set Hessian Threshold to 400
surf = cv2.SURF(400)
#Find keypoints and descriptors directly
kp1, des1 = surf.detectAndCompute(img1,None)
kp2, des2 = surf.detectAndCompute(img2,None)

# Check present Hessian threshold
print surf.hessianThreshold

# We set it to some 50000. Remember, it is just for representing in picture.
# In actual cases, it is better to have a value 300-500
surf.hessianThreshold = 1000

# Again compute keypoints and check its number.
kp1, des1 = surf.detectAndCompute(img1,None)
kp2, des2 = surf.detectAndCompute(img2,None)


# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# Match descriptors.
matches = bf.match(des1,des2)

# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)

# Draw first 10 matches.
img3 = drawMatches(img1,kp1,img2,kp2,matches[:10])

plt.imshow(img3),plt.show()

400.0


(<matplotlib.image.AxesImage at 0xf03cb70>, None)

In [8]:
#!/usr/bin/env python

'''
Uses SURF to match two images.
Based on the sample code from opencv:
  samples/python2/find_obj.py
USAGE
  find_obj.py <image1> <image2>
'''

import numpy as np
import cv2

import sys

###############################################################################
# Image Matching
###############################################################################

def match_images(img1, img2):
    """Given two images, returns the matches"""
    detector = cv2.SURF(400, 5, 5)
    matcher = cv2.BFMatcher(cv2.NORM_L2)

    kp1, desc1 = detector.detectAndCompute(img1, None)
    kp2, desc2 = detector.detectAndCompute(img2, None)
    #print 'img1 - %d features, img2 - %d features' % (len(kp1), len(kp2))

    raw_matches = matcher.knnMatch(desc1, trainDescriptors = desc2, k = 2) #2
    kp_pairs = filter_matches(kp1, kp2, raw_matches)
    return kp_pairs

def filter_matches(kp1, kp2, matches, ratio = 0.75):
    mkp1, mkp2 = [], []
    for m in matches:
        if len(m) == 2 and m[0].distance < m[1].distance * ratio:
            m = m[0]
            mkp1.append( kp1[m.queryIdx] )
            mkp2.append( kp2[m.trainIdx] )
    kp_pairs = zip(mkp1, mkp2)
    return kp_pairs
    
    
###############################################################################
# Match Diplaying
###############################################################################

def explore_match(win, img1, img2, kp_pairs, status = None, H = None):
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    vis = np.zeros((max(h1, h2), w1+w2), np.uint8)
    vis[:h1, :w1] = img1
    vis[:h2, w1:w1+w2] = img2
    vis = cv2.cvtColor(vis, cv2.COLOR_GRAY2BGR)

    if H is not None:
        corners = np.float32([[0, 0], [w1, 0], [w1, h1], [0, h1]])
        corners = np.int32( cv2.perspectiveTransform(corners.reshape(1, -1, 2), H).reshape(-1, 2) + (w1, 0) )
        cv2.polylines(vis, [corners], True, (255, 255, 255))

    if status is None:
        status = np.ones(len(kp_pairs), np.bool_)
    p1 = np.int32([kpp[0].pt for kpp in kp_pairs])
    p2 = np.int32([kpp[1].pt for kpp in kp_pairs]) + (w1, 0)

    green = (0, 255, 0)
    red = (0, 0, 255)
    white = (255, 255, 255)
    kp_color = (51, 103, 236)
    for (x1, y1), (x2, y2), inlier in zip(p1, p2, status):
        if inlier:
            col = green
            cv2.circle(vis, (x1, y1), 2, col, -1)
            cv2.circle(vis, (x2, y2), 2, col, -1)
        else:
            col = red
            r = 2
            thickness = 3
            cv2.line(vis, (x1-r, y1-r), (x1+r, y1+r), col, thickness)
            cv2.line(vis, (x1-r, y1+r), (x1+r, y1-r), col, thickness)
            cv2.line(vis, (x2-r, y2-r), (x2+r, y2+r), col, thickness)
            cv2.line(vis, (x2-r, y2+r), (x2+r, y2-r), col, thickness)
    vis0 = vis.copy()
    for (x1, y1), (x2, y2), inlier in zip(p1, p2, status):
        if inlier:
            cv2.line(vis, (x1, y1), (x2, y2), green)

    cv2.imshow(win, vis)


  
def draw_matches(window_name, kp_pairs, img1, img2):
    """Draws the matches for """
    mkp1, mkp2 = zip(*kp_pairs)
    
    p1 = np.float32([kp.pt for kp in mkp1])
    p2 = np.float32([kp.pt for kp in mkp2])
    
    if len(kp_pairs) >= 4:
        H, status = cv2.findHomography(p1, p2, cv2.RANSAC, 5.0)
        #print '%d / %d  inliers/matched' % (np.sum(status), len(status))
    else:
        H, status = None, None
        #print '%d matches found, not enough for homography estimation' % len(p1)
    
    if len(p1):
        explore_match(window_name, img1, img2, kp_pairs, status, H)

###############################################################################
# Test Main
###############################################################################

if __name__ == '__main__':
    """Test code: Uses the two specified"""
    if len(sys.argv) < 3:
        print "No filenames specified"
        print "USAGE: find_obj.py <image1> <image2>"
        sys.exit(1)
    
#     fn1 = sys.argv[1]
#     fn2 = sys.argv[2]

    img1 = cv2.imread('simitsu4.jpg', 0)
    img2 = cv2.imread('simitsu3.jpg', 0)
    
    if img1 is None:
        print 'Failed to load fn1:', fn1
        sys.exit(1)
        
    if img2 is None:
        print 'Failed to load fn2:', fn2
        sys.exit(1)

    kp_pairs = match_images(img1, img2)
    
    if kp_pairs:
        draw_matches('find_obj', kp_pairs, img1, img2)
        cv2.waitKey()
        cv2.destroyAllWindows()    
else:
        print "No matches found"